In [1]:
import pandas as pd
import numpy as np

import os


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
TRANSFORM_FOLDER = '../../DATA/TRANSFORM/entity_details/'
FINAL_FOLDER = '../../DATA/FINAL'

In [3]:
financial_ratios = pd.read_csv('../../DATA/MODEL/financial_ratios_df.csv')

C:\Users\matus.rohal\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3433: DtypeWarning: Columns (110) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
financial_ratios

,entity_id,year,month,financial_report_id,financial_report_template_id,total_assets,non_current_assets,non_current_intangible_assets,non_current_tangible_assets,non_current_financial_assets,current_assets,inventories,non_current_receivables,current_receivables,financial_assets,current_financial_assets,cash_and_bank_balances,total_equity_and_liabilities,equity,share_capital,share_premium,net_profit_of_previous_years,net_profit_after_tax,liabilities,non_current_liabilities,long_term_provisions,long_term_bank_loans,current_liabilities,short_term_provisions,current_bank_loans,short_term_financial_assistance,operating_revenues,sales_from_the_merchandise,sales_from_the_own_products_and_services,sales_from_the_other,other_revenues_from_operating_activities,operating_costs,costs_of_merchandise_sold,consumed_materials_energy_non_inventory_supplies,services,personnel_costs,taxes_and_fees,depreciation,remaining_cost_of_sold_long_term_assets_and_materials,adjustments_to_receivables,other_costs_of_operating_activities,operating_result,value_added,revenues_from_financial_activities,cost_of_financial_activities,...,YIE,LABOR,L3,L2,L1,CF_CL,CASH_TA,SAL_TA,TL_SAL,INV_COST,INV_SAL,CC_SAL,TA_SAL,TL_TA,CF_TL,CL_TA,NCL_TA,EQ_TL,EQ_TA,EBIT_INT,CL_CC,ROE,EAT_TA,ROA_BRUTTO,CF_TA,CF_SAL,ROS,EAT_YIE,ROI,ROA_NETTO,LABOR_PRODUCTIVITY,financial_statement_id,financial_report_order_number,entity_ico,period_length,consolidated,approved_date,type,entity_name,establishment_date,cancellation_date,sknace_code,entity_state,legal_form_code,org_size_code,ownership_category_code,entity_consolidated,sknace_division_name,sknace_division,sknace_subcategory
0,4.0,2014.0,1.0,4644773,699.0,11200988.0,1946914.0,14341.0,106669.0,1825904.0,8787618.0,16429.0,953600.0,2578606.0,5238983.0,0.0,5238983.0,11200988.0,2112122.0,39900.0,1143260.0,-78935.0,1007897.0,8657266.0,3232.0,40297.0,0.0,4658457.0,3906138.0,49142.0,0.0,18580340.0,0.0,18416986.0,0.0,163354.0,18172171.0,0.0,117382.0,14389476.0,3384185.0,2946.0,55386.0,0.0,173287.0,49509.0,408169.0,3910128.0,780965.0,73442.0,...,19361305.0,3384185.0,1.886380,1.678150,1.124618,0.251388,0.467725,1.658813,0.465937,0.325467,0.318317,0.138781,217.022707,0.772902,0.135271,0.415897,0.000289,0.243971,0.188566,251.942870,1.806580,0.477196,0.089983,0.099607,0.104551,0.063028,0.054245,0.052057,0.090380,0.089983,0.182138,2382229.0,1.0,603783.0,12.0,NaN,2015-07-07,Riadna,"NESS Slovensko, a.s.",1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0
1,4.0,2015.0,1.0,5382540,699.0,20862929.0,116951.0,14800.0,101401.0,750.0,20276962.0,6792.0,545698.0,14037315.0,5687157.0,0.0,5687157.0,20862929.0,13229589.0,39900.0,1143260.0,5510185.0,6536244.0,7184264.0,2890.0,0.0,0.0,6153737.0,1027637.0,0.0,0.0,17608945.0,0.0,17435113.0,0.0,173832.0,11551924.0,0.0,106101.0,13052044.0,3581817.0,3727.0,-3290982.0,0.0,-46425.0,-1854358.0,6057021.0,4276968.0,11964908.0,10790551.0,...,29573853.0,3581817.0,3.295065,3.205284,0.924179,0.640326,0.272596,0.844030,0.407989,0.211663,0.138857,0.797170,426.524953,0.344355,0.548476,0.294960,0.000139,1.841468,0.634119,2486.009622,0.438384,0.494063,0.313295,0.346614,0.188871,0.223772,0.371189,0.221014,0.313434,0.313295,0.203409,2968194.0,1.0,603783.0,12.0,NaN,2016-09-28,Riadna,"NESS Slovensko, a.s.",1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0
2,4.0,2016.0,1.0,5665728,699.0,6518310.0,113674.0,13738.0,99936.0,0.0,5991927.0,5894.0,316700.0,3840942.0,1828391.0,0.0,1828391.0,6518310.0,1178778.0,79800.0,1143260.0,0.0,-44282.0,5120402.0,3568.0,0.0,0.0,4859422.0,257412.0,0.0,0.0,12878215.0,0.0,12713547.0,608.0,164060.0,12564620.0,0.0,87183.0,9060953.0,3476051.0,4829.0,55984.0,3066.0,-165814.0,42368.0,313595.0,3565411.0,36391.0,158830.0,...,12914606.0,3476051.0,1.233053,1.166668,0.376257,0.050858,0.280501,1.975698,0.397602,0.168874,0.164762,0.298251,182.214041,0.785541,0.048266,0.745503,0.000547,0.230212,0.180841,6.000288,1.265

## Joining financial issues indicators

In [7]:
# pomer VI a zavazkov je mensi ako 8, t.j EQ_TL < 0.08
# 2016 : 0.04, 2017 : 0.06, 2018 : 0.08, 
# EQ < 0
# L3 < 1

In [5]:
# def process_issues(row):
#     EQ_TL = row.EQ_TL
#     EQ = row.EQ
#     L3 = row.L3
#     EAT = row.EAT
#     y = row.year

#     if y <= 2016: threat_criterion = 0.04
#     elif y == 2017: threat_criterion = 0.06
#     else: threat_criterion = 0.08

#     if EQ_TL < threat_criterion and EQ < 0 and L3 < 1 and EAT < 0:
#         return 1
#     return 0
    

In [6]:
# legal_financial_issues_criterion = []
# for i, row in financial_ratios.iterrows():
#     legal_financial_issues_criterion.append(
#         (row.entity_ico,
#         row.year,
#         process_issues(row))      
#     )

# financial_issues_criterion_df = pd.DataFrame(legal_financial_issues_criterion, columns=['entity_ico', 'year', 'RUZ_indirect_criterion'])
# display(financial_issues_criterion_df)

In [7]:
# financial_issues_criterion_info = financial_issues_criterion_df.query('RUZ_indirect_criterion > 0').drop(columns='RUZ_indirect_criterion')
# financial_issues_criterion_info['event_type'] = 'RUZ_indirect_criterion'

In [8]:
files = [
 'insolvency_register.csv',
 'vat_registration_cancellation.csv']

In [9]:
data = dict()
data['insolvency_register'] = pd.read_csv(TRANSFORM_FOLDER + 'insolvency_register.csv')
data['vat_registration_cancellation'] = pd.read_csv(TRANSFORM_FOLDER + 'vat_registration_cancellation.csv')

In [16]:
data['insolvency_register']['proceeding_type'].value_counts()

konkurz                       5813
restrukturalizacia             691
starykonkurz                   472
oddlzeniekonkurz                87
inekonanie                      40
vyrovnanie                      16
oddlzeniesplatkovykalendar       4
Name: proceeding_type, dtype: int64

In [11]:
insolvency_info = data['insolvency_register'].copy()
insolvency_info['proceeding_type'] = insolvency_info['proceeding_type'].apply(lambda x: 'ostatne_konania' if x not in ['konkurz', 'restrukturalizacia'] else x)
insolvency_info

insolvency_info.drop_duplicates(['entity_ico', 'proceeding_type', 'proceeding_month'], inplace=True)

insolvency_info = insolvency_info.sort_values('proceeding_month', ascending=False).groupby(['entity_ico', 'proceeding_type', 'proceeding_month']).tail(1)

insolvency_info.rename(columns={'proceeding_type': 'event_type', 'proceeding_month': 'year'}, inplace=True)

insolvency_info = insolvency_info[['entity_ico', 'year', 'event_type']]

insolvency_info['event_type'] = insolvency_info['event_type'].apply(lambda x: 'RU_' + x)
insolvency_info



,entity_ico,year,event_type
0,53051505.0,2023,RU_konkurz
276,45380058.0,2023,RU_konkurz
287,45257337.0,2023,RU_konkurz
286,35964065.0,2023,RU_konkurz
285,44353260.0,2023,RU_konkurz
...,...,...,...
7033,31105424.0,1993,RU_ostatne_konania
7032,31655785.0,1993,RU_ostatne_konania
6995,598321.0,1993,RU_ostatne_konania
7019,31412874.0,1993,RU_ostatne_konania


In [20]:
insolvency_info = data['insolvency_register'].copy().drop_duplicates(['entity_ico', 'proceeding_month'])[['entity_ico', 'proceeding_month']]
insolvency_info.columns = ['entity_ico', 'year']
insolvency_info['event_type'] = 'RU_event'

insolvency_info


,entity_ico,year,event_type
0,53051505.0,2023,RU_event
1,53099231.0,2023,RU_event
2,35709596.0,2023,RU_event
3,45423873.0,2023,RU_event
4,44031114.0,2023,RU_event
...,...,...,...
7118,43291546.0,2017,RU_event
7119,33946477.0,2017,RU_event
7120,50365738.0,2017,RU_event
7121,32576391.0,2017,RU_event


In [22]:
vat_registration_cancellation_info = data['vat_registration_cancellation'].copy()
# vat_registration_cancellation_info

vat_registration_cancellation_info['entity_ico'] = pd.to_numeric(vat_registration_cancellation_info['entity_ico'], errors = 'coerce')
vat_registration_cancellation_info.query('entity_ico == entity_ico', inplace=True)
vat_registration_cancellation_info = vat_registration_cancellation_info[['entity_ico', 'vat_cancellation_year']]
vat_registration_cancellation_info['vat_cancellation_year'] = pd.to_numeric(vat_registration_cancellation_info['vat_cancellation_year'])
vat_registration_cancellation_info['event_type'] = 'DPHZ_vat_registration_cancelled'
vat_registration_cancellation_info.rename(columns = {'vat_cancellation_year': 'year'}, inplace=True)
vat_registration_cancellation_info

,entity_ico,year,event_type
0,37712080.0,2020,DPHZ_vat_registration_cancelled
1,37257455.0,2018,DPHZ_vat_registration_cancelled
2,37713256.0,2013,DPHZ_vat_registration_cancelled
3,35125730.0,2020,DPHZ_vat_registration_cancelled
4,37338048.0,2020,DPHZ_vat_registration_cancelled
...,...,...,...
4749,2083477.0,2018,DPHZ_vat_registration_cancelled
4750,50635808.0,2023,DPHZ_vat_registration_cancelled
4751,6199917.0,2020,DPHZ_vat_registration_cancelled
4752,53714521.0,2023,DPHZ_vat_registration_cancelled


In [23]:
valid_ico_list = list(financial_ratios['entity_ico'].unique())

In [26]:
entity_issues_events_long = (pd.concat(
    [
         insolvency_info,
         vat_registration_cancellation_info
    ], axis = 0))
entity_issues_events_long.query('entity_ico in @valid_ico_list and year >= 2014', inplace=True)

entity_issues_events_long['event_type'].value_counts()

DPHZ_vat_registration_cancelled    3608
RU_event                           3337
Name: event_type, dtype: int64

In [27]:
entity_issues_events_long['indicator'] = 1
entity_issues_events_long.to_csv('../../DATA/TRANSFORM/issues_events.csv', index = False)

In [28]:

entity_issues_indicators_1y_off = entity_issues_events_long.copy()
entity_issues_indicators_2y_off = entity_issues_events_long.copy()
entity_issues_indicators_3y_off = entity_issues_events_long.copy()

entity_issues_indicators_1y_off['year'] = entity_issues_indicators_1y_off['year'] - 1
entity_issues_indicators_1y_off['event_type'] = entity_issues_indicators_1y_off['event_type'].apply(lambda x: x+'_1y_off')

entity_issues_indicators_2y_off['year'] = entity_issues_indicators_2y_off['year'] - 2
entity_issues_indicators_2y_off['event_type'] = entity_issues_indicators_2y_off['event_type'].apply(lambda x: x+'_2y_off')

entity_issues_indicators_3y_off['year'] = entity_issues_indicators_3y_off['year'] - 3
entity_issues_indicators_3y_off['event_type'] = entity_issues_indicators_3y_off['event_type'].apply(lambda x: x+'_3y_off')


entity_issues_indicators = (pd.concat(
    [
        entity_issues_events_long.copy(),
        entity_issues_indicators_1y_off,
        entity_issues_indicators_2y_off,
        entity_issues_indicators_3y_off
    ], axis=0
))

entity_issues_events = pd.pivot_table(entity_issues_indicators, index = ['entity_ico', 'year'], columns = 'event_type', values='indicator', fill_value=0)
entity_issues_events.reset_index(inplace=True)
entity_issues_events

event_type,entity_ico,year,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,DPHZ_vat_registration_cancelled_3y_off,RU_event,RU_event_1y_off,RU_event_2y_off,RU_event_3y_off
0,671.0,2015,0,0,0,0,0,0,0,1
1,671.0,2016,0,0,0,0,0,0,1,0
2,671.0,2017,0,0,0,0,0,1,0,0
3,671.0,2018,0,0,0,0,1,0,0,0
4,188034.0,2011,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
26974,54300681.0,2023,1,0,0,0,0,0,0,0
26975,54303575.0,2020,0,0,0,1,0,0,0,0
26976,54303575.0,2021,0,0,1,0,0,0,0,0
26977,54303575.0,2022,0,1,0,0,0,0,0,0


In [29]:
ISSUES_TYPES = ['DPHZ_vat_registration_cancelled', 'RU_event']

In [30]:
def process_entity_issues_2y_off(row, issue_type):
        return max(row[issue_type+'_1y_off'], row[issue_type+'_2y_off'])

def process_entity_issues_3y_off(row, issue_type):
        return max(row[issue_type+'_1y_off'], row[issue_type+'_2y_off'], row[issue_type+'_3y_off'])

for issue_type in ISSUES_TYPES:
    entity_issues_events[issue_type+'_2y_off'] = entity_issues_events.apply(lambda x: process_entity_issues_2y_off(x, issue_type), axis = 1)
    entity_issues_events[issue_type+'_3y_off'] = entity_issues_events.apply(lambda x: process_entity_issues_3y_off(x, issue_type), axis = 1)

entity_issues_events

event_type,entity_ico,year,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,DPHZ_vat_registration_cancelled_3y_off,RU_event,RU_event_1y_off,RU_event_2y_off,RU_event_3y_off
0,671.0,2015,0,0,0.0,0.0,0,0,0.0,1.0
1,671.0,2016,0,0,0.0,0.0,0,0,1.0,1.0
2,671.0,2017,0,0,0.0,0.0,0,1,1.0,1.0
3,671.0,2018,0,0,0.0,0.0,1,0,0.0,0.0
4,188034.0,2011,0,0,0.0,0.0,0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
26974,54300681.0,2023,1,0,0.0,0.0,0,0,0.0,0.0
26975,54303575.0,2020,0,0,0.0,1.0,0,0,0.0,0.0
26976,54303575.0,2021,0,0,1.0,1.0,0,0,0.0,0.0
26977,54303575.0,2022,0,1,1.0,1.0,0,0,0.0,0.0


In [37]:
display(financial_ratios.shape[0])

financial_ratios = financial_ratios.merge(entity_issues_events, on = ['entity_ico', 'year'], how = 'left')
display(financial_ratios.shape[0])


1687865

1687865

In [39]:
financial_ratios_copy = None

In [40]:
financial_ratios['cancellation_year'] = pd.to_datetime(financial_ratios['cancellation_date'], errors='coerce').dt.year

In [41]:
def ruz_cancelled(row):
    if row.year >= row.cancellation_year: return 1
    return 0

def ruz_cancelled_1y_off(row):
    if row.year + 1 >= row.cancellation_year: return 1
    return 0

def ruz_cancelled_2y_off(row):
    if row.year + 2 >= row.cancellation_year: return 1
    return 0


def ruz_cancelled_3y_off(row):
    if row.year + 3 >= row.cancellation_year: return 1
    return 0


financial_ratios['RUZ_cancelled'] = financial_ratios.apply(ruz_cancelled, axis=1)
financial_ratios['RUZ_cancelled_1y_off'] = financial_ratios.apply(ruz_cancelled_1y_off, axis=1)
financial_ratios['RUZ_cancelled_2y_off'] = financial_ratios.apply(ruz_cancelled_2y_off, axis=1)
financial_ratios['RUZ_cancelled_3y_off'] = financial_ratios.apply(ruz_cancelled_3y_off, axis=1)

In [42]:
financial_ratios.shape

(1687865, 133)

In [43]:
for col in ['DPHZ_vat_registration_cancelled',
 'DPHZ_vat_registration_cancelled_1y_off',
 'DPHZ_vat_registration_cancelled_2y_off',
 'DPHZ_vat_registration_cancelled_3y_off',
 'RU_event',
 'RU_event_1y_off',
 'RU_event_2y_off',
 'RU_event_3y_off',
 'RUZ_cancelled',
 'RUZ_cancelled_1y_off',
 'RUZ_cancelled_2y_off',
 'RUZ_cancelled_3y_off']:
    financial_ratios[col] = financial_ratios[col].fillna(0)



In [44]:
financial_ratios.to_csv('../../DATA/MODEL/transformed_set.csv', index = False)

In [45]:
entity_issues_indicators = entity_issues_indicators.query('entity_ico > 0')[['entity_ico', 'year'] + sorted(list(entity_issues_indicators.columns)[2:])].sort_values(['entity_ico', 'year'])

In [46]:
entity_issues_indicators.to_csv('../../DATA/TRANSFORM/issues_indicators.csv', index = False)